In [5]:
import os
import os.path
import numpy as np

# clean content
def clean_content(imdb_str):
    
    # imdb_str = ' ' + imdb_str + ' '
    imdb_word_split = []
    count_flag      = 0
    imdb_str        = imdb_str.lower() # 大小写转换
    for i in range(len(imdb_str) - 1):

        if(imdb_str[i] is ' ')and(imdb_str[i+1] is not ' ')and(count_flag is 0):
            start_index = i+1
            count_flag  = 1

        if(imdb_str[i] is not ' ')and(imdb_str[i+1] is ' ')and(count_flag is 1):
            end_index = i+1
            imdb_word_split.append(imdb_str[start_index:end_index])
            count_flag = 0
    return imdb_word_split

# file
train_file_name = 'train_5500.label.txt'
train_file  = []
train_label = []
with open(train_file_name,'r') as f:  #注意open 和 codecs.opem
    lines = f.readlines()
    for i in range(len(lines)):
        lines_clean = lines[i].rstrip('\n')
        cut_index   = lines_clean.find(':')
        train_label.append(lines_clean[0:cut_index])
        train_file.append(' ' + lines_clean[cut_index+1:])  # 末尾的?,.这些暂时去掉
        
test_file_name  = 'TREC_10.label.txt'
test_file  = []
test_label = []

with open(test_file_name,'r') as f:  #注意open 和 codecs.opem
    lines = f.readlines()
    for i in range(len(lines)):
        lines_clean = lines[i].rstrip('\n')
        cut_index   = lines_clean.find(':')
        test_label.append(lines_clean[0:cut_index])
        test_file.append(' ' + lines_clean[cut_index+1:])

# label
class_num = []
for i in range(len(train_label)):
    if train_label[i] not in class_num:
        class_num.append(train_label[i])
        
trec_train_label = []
trec_train_split = []
for i in range(len(train_label)):
    trec_train_label.append(class_num.index(train_label[i]))
    trec_train_split.append(clean_content(train_file[i]))
    
trec_test_label = []
trec_test_split = []
for i in range(len(test_label)):
    trec_test_label.append(class_num.index(test_label[i]))
    trec_test_split.append(clean_content(test_file[i]))

# calculate vab
trec_train_list = train_file
trec_test_list  = test_file
trec_vab_count  = {}

for i in range(len(trec_train_split)):
    trec_str = trec_train_split[i]
    for word_index in range(len(trec_str)):
        if (not trec_vab_count.has_key(trec_str[word_index])):
            trec_vab_count.update({trec_str[word_index]: 1})
        else:
            trec_vab_count[trec_str[word_index]] = trec_vab_count[trec_str[word_index]] + 1 
                
del_stop_word = False
if del_stop_word is True:
    # stop word list
    stop_word_list = []
    with open('/home/xiaosucheng/Desktop/IMDB/IMDB_Read_Data/imdb_stop_word.txt','r') as f:  #注意open 和 codecs.opem
        line = f.readlines()
        for i in range(len(line)):
            stop_word_list.append(line[i].rstrip('\n'))

    for i in range(len(stop_word_list)):
        if trec_vab_count.has_key(stop_word_list[i]):
            trec_vab_count.pop(stop_word_list[i])
    print 'delete stop word' 

# sort vab
trec_vab                  = {}
trec_vab_list_orgin       = trec_vab_count.keys()   # list
trec_vab_count_list_orgin = trec_vab_count.values() # list
trec_vab_list             = []
trec_vab_count_list       = []

trec_vab_index            = np.argsort(-np.array(trec_vab_count_list_orgin))  # np.argsort(-x) 降序排序
for i in range(len(trec_vab_list_orgin)):
    trec_vab.update({trec_vab_list_orgin[trec_vab_index[i]]: i + 1})    # 0 给stop_word　所以+1
    trec_vab_list.append(trec_vab_list_orgin[trec_vab_index[i]])
    trec_vab_count_list.append(trec_vab_count_list_orgin[trec_vab_index[i]])

print 'calculate vocabulary finished'

vab_size = 8000
# word2index 如果不含有该词或者index超过vab_size,则置成０
stop_flag = 1
trec_train_list_index = []
for i in range(len(trec_train_split)):
    trec_train_list_index.append([-1])  # 添加一个新的list
    trec_str = trec_train_split[i]      # 把对应的str取出来
    for word_index in range(len(trec_str)):
        if (not trec_vab.has_key(trec_str[word_index])) or (trec_vab[trec_str[word_index]] >= vab_size): # 出现为０的值
            if (word_index > 0 and stop_flag is not 0):  # 连续为0的值只取一个0
                trec_train_list_index[i].append(0)
                stop_flag = 0
        else:
            trec_train_list_index[i].append(trec_vab[trec_str[word_index]])
            stop_flag = 1
    trec_train_list_index[i].pop(0)  # 删除第一个元素
print 'train word2index finished'

stop_flag = 1
trec_test_list_index = []
for i in range(len(trec_test_split)):
    trec_test_list_index.append([-1])
    trec_str = trec_test_split[i]
    for word_index in range(len(trec_str)):
        if (not trec_vab.has_key(trec_str[word_index])) or (trec_vab[trec_str[word_index]] >= vab_size):
            if (word_index > 0 and stop_flag is not 0):
                trec_test_list_index[i].append(0)
                stop_flag = 0 
        else:
            trec_test_list_index[i].append(trec_vab[trec_str[word_index]])
            stop_flag = 1
    trec_test_list_index[i].pop(0)
print 'test word2index finished'

# imdb_vab_list 　　　　　 词表的词
# imdb_vab_count_list 　 词表词的词频

import cPickle
TREC = {}
TREC['Vocabulary']       = trec_vab_list
TREC['Vab_count']        = trec_vab_count_list
TREC['Vab_Size']         = vab_size
TREC['Label']            = class_num
TREC['Train_Origin']     = trec_train_list
TREC['Train_Label']      = trec_train_label
TREC['Train_Word_Split'] = trec_train_split
TREC['Train_Word2Index'] = trec_train_list_index
TREC['Test_Origin']      = trec_test_list
TREC['Test_Label']       = trec_test_label
TREC['Test_Word_Split']  = trec_test_split
TREC['Test_Word2Index']  = trec_test_list_index
cPickle.dump(TREC, open('./TREC_8k.pkl','wb'))
print 'save finished'   

calculate vocabulary finished
train word2index finished
test word2index finished
save finished


In [11]:
import numpy as np
import scipy.io as sio

# ELEC['Train_BOW'] = np.zeros([30000, 25000])
# for i in range(len(ELEC['Train_Word2Index'])):
    
#     elec_str = ELEC['Train_Word2Index'][i]
#     for j in range(len(elec_str)):
        
#         ELEC['Train_BOW'][elec_str[j],i] = ELEC['Train_BOW'][elec_str[j],i] + 1
        
# ELEC['Test_BOW'] = np.zeros([30000, 25000])
# for i in range(len(ELEC['Test_Word2Index'])):
    
#     elec_str = ELEC['Test_Word2Index'][i]
#     for j in range(len(elec_str)):
        
#         ELEC['Test_BOW'][elec_str[j],i] = ELEC['Test_BOW'][elec_str[j],i] + 1        
        
    
sio.savemat('TREC_8k.mat',{'TREC_Train_Origin':     TREC['Train_Origin'],
                           'TREC_Test_Origin':      TREC['Test_Origin'],              
                           'TREC_Train_Word2Index': TREC['Train_Word2Index'],
                           'TREC_Vab':              TREC['Vocabulary'],
                           'TREC_Vab_Len':          TREC['Vab_Size'],
                           'TREC_Test_Word2Index':  TREC['Test_Word2Index'],
                           'TREC_Train_Label':      list(TREC['Train_Label']),
                           'TREC_Test_Label':       list(TREC['Test_Label'])
                           })

print 'save mat finished'

save mat finished
